<a href="https://colab.research.google.com/github/hgrigolli/compiladores/blob/main/COMPILADORES_AULA_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPILADORES - AULA 04**

**Prof. Luciano Silva**

**Aluno: Henrique Nóbrega Grigolli - 41821661 - 06N**


**OBJETIVOS DA AULA:**



*   Revisar o processo de análise sintática
*   Implementar um analisador sintático para a linguagem TINY-C



In [1]:
!pip install rply

**REVISÃO DO PROCESSO DE ANÁLISE SINTÁTICA**

Na nossa última aula, implementamos um analisador sintático completo para o comando de atribuição com expressões ariméticas envolvendo números inteiros sem sinal:

\<atrib\>::= ID "=" \<expression\>

\<expression\> ::= NUMBER

       | \<expression\> "+" \<expression\>
 
       | \<expression\> "-" \<expression\>
 
       | \<expression\> "*" \<expression\>
 
       | \<expression\> "/" \<expression\>
 
       | "(" <expression> ")"

O primeiro passo foi implementar um analisador léxico para esta gramática, mostrado abaixo:


In [2]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('EQUALS', r'=')
lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.ignore('\s+')

lexer = lg.build()

O segundo passo foi implementar as classes em Python para representar os nós da árvore sintática gerada pelo analisador sintático:

In [3]:
from rply.token import BaseBox



class BinaryOp(BaseBox):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    def eval(self):
        return self.left.eval() + self.right.eval()

class Sub(BinaryOp):
    def eval(self):
        return self.left.eval() - self.right.eval()

class Mul(BinaryOp):
    def eval(self):
        return self.left.eval() * self.right.eval()
class Number(BaseBox):
    def __init__(self, value):
        self.value = value

    def eval(self):
        return self.value
class Div(BinaryOp):
    def eval(self):
        return self.left.eval() / self.right.eval()

class Attrib(BaseBox):
    def __init__(self, id, expression):
        self.id = id
        self.expression = expression

Finalmente, foi implementado o analisado sintático para o comando de atribuição:

In [23]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV','ID','EQUALS'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])    
    ]
)

# regra <atrib>::= ID "=" <expression>

@pg.production('atrib : ID EQUALS expression')
def attrib(p):
  return Attrib(p[0].getstr(),p[2])

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

Realizamos um teste com um comando de atribuição:

In [54]:
arvore=parser.parse(lexer.lex('x=1+2*3'))
print(arvore)
print(arvore.id)
print(arvore.expression.eval())

ParsingError: ignored

**EXERCÍCIO**

*Implementar e testar um analisador sintático para a gramática da linguagem TINY-C. Todos os nós da árvore sintática devem implementar o método print que, quando invocado, deve mostrar todo o conteúdo armazenado nos seus atributos.*




program ::= function (function)*

function ::= type_specifier id ‘(‘ param_decl_list ‘)’ compound_stmt

type_specifier ::= int | char

param_decl_list ::= parameter_decl (‘,’ parameter_decl )*

param_decl ::= type_specifier id

compound_stmt ::= ‘{‘ (var_decl stmt)? ‘}’

var_decl ::= type_specifier var_decl_list ‘;’

var_decl_list ::= variable_id ( ‘,’ variable_id)*

variable_id ::= id ( ‘=’ expr )? | id '[' num ']'

stmt ::= compound_stmt | cond_stmt | while_stmt | break ‘;’ | continue ‘;’ | return expr ‘;’ | readint ‘(‘ id ‘)’ ‘;’ | writeint ‘(‘ expr ‘)’ ‘;’

cond_stmt ::= if ‘(‘ expr ‘)’ stmt (else stmt)?

while_stmt ::= while ‘(‘ expr ‘)’ stmt

expr ::= id ‘=’ expr | condition

condition ::= disjunction | disjunction ‘?’ expr ‘:’ condition

disjunction ::= conjunction | disjunction ‘||’ conjunction

conjunction ::= comparison | conjunction ‘&&’ comparison

comparison ::= relation | relation ‘==’ relation

relation ::= sum | sum (‘<’ | ‘>’) sum

sum ::= sum ‘+’ term | sum ‘-’ term | term

term ::= term ‘*’ factor | term ‘/’ factor | term ‘%’ factor | factor

factor ::= ‘!’ factor | ‘-’ factor | primary

primary ::= num | charconst | id | ‘(‘ expr ‘)’ 

A implementação do analisador léxico está disponível abaixo:

In [55]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('INT', r'int')
lg.add('CHAR', r'char')
lg.add('BREAK', r'break')
lg.add('CONTINUE', r'continue')
lg.add('RETURN', r'return')
lg.add('READINT', r'readint')
lg.add('WRITEINT', r'writeint')
lg.add('IF', r'if')
lg.add('ELSE', r'else')
lg.add('WHILE', r'while')
lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')
lg.add('OPEN_COL', r'\[')
lg.add('CLOSE_COL', r'\]')
lg.add('VIRG', r'\,')
lg.add('OPEN_CH', r'\{')
lg.add('CLOSE_CH', r'\}')
lg.add('PVIRG', r'\;')
lg.add('COMPEQUALS', r'==')
lg.add('COMPMAIOR', r'\>')
lg.add('COMPMENOR', r'\<')
lg.add('EQUALS', r'=')
lg.add('INTERROG', r'\?')
lg.add('DOISP', r'\:')
lg.add('DISJ', r'\|\|')
lg.add('CONJ', r'&&')
lg.add('NOT', r'\!')
lg.add('NUMBER', r'\d+')
lg.add('CHARCONST', r'\'\S\'')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('MOD', r'\%')

lg.ignore('\s+')

lexer = lg.build()

In [131]:
#@title Classes
#implemente suas classes dos nós da árvore sintática aqui
class Program(BaseBox):
    def __init__(self, function, manyFunction=None):
      self.function = function
      self.manyFunction = manyFunction

    def print(self):
      self.function.print()
      if self.manyFunction is not None:
        self.manyFunction.print()

class Function(BaseBox):
  def __init__(self, typeSpecifier, id, paramDeclList, compoundStmt):
    self.typeSpecifier = typeSpecifier
    self.id = id
    self.paramDeclList = paramDeclList
    self.compoundStmt = compoundStmt

  def print(self):
    print(self.typeSpecifier)
    print(self.id)
    self.paramDeclList.print()
    self.compoundStmt.print()

class ParamDeclList(BaseBox):
  def __init__(self, parameterDecl, manyParameterDecl=None):
    self.parameterDecl = parameterDecl
    self.manyParameterDecl = manyParameterDecl
  
  def print(self):
    self.parameterDecl.print()
    if self.manyParameterDecl is not None:
      self.manyParameterDecl.print()

class ParamDecl(BaseBox):
  def __init__(self, typeSpecifier, id):
    self.typeSpecifier = typeSpecifier
    self.id = id

  def print(self):
    print(self.typeSpecifier)
    print(self.id)

class CompoundStmt(BaseBox):
  def __init__(self, varDecl, stmt):
    self.varDecl = varDecl
    self.stmt = stmt

  def print(self):
    self.varDecl.print()
    self.stmt.print()

class VarDecl(BaseBox):
  def __init__(self, typeSpecifier, varDeclList):
    self.typeSpecifier = typeSpecifier
    self.varDeclList = varDeclList

  def print(self):
    print(self.typeSpecifier)
    self.varDeclList.print()

class VarDeclList(BaseBox):
  def __init__(self, variableId, manyVariableId=None):
    self.variableId = variableId
    self.manyVariableId = manyVariableId
  
  def print(self):
    self.variableId.print()
    if self.manyVariableId is not None:
      self.manyVariableId.print()

class VariableId(BaseBox):
  def __init__(self, id):
    self.id = id

  def print(self):
    print(self.id)

  
class VariableIdExpr(VariableId):
  def __init__(self, id, expr):
    super().__init__(id)
    self.expr = expr

  def print(self):
    super().print()
    self.expr.print()

class VariableIdNum(VariableId):
  def __init__(self, id, num):
    super().__init__(id)
    self.num = num

  def print(self):
    super().print()
    print(self.num)

class Stmt(BaseBox):
  def __init__(self):
    pass

  def print(self):
    pass

class StmtCompound(Stmt):
  def __init__(self, compoundStmt):
    self.compoundStmt = compoundStmt

  def print(self):
    self.compoundStmt.print()

class CondStmt(Stmt):
  def __init__(self, expr, ifStmt, elseStmt=None):
    self.expr = expr
    self.ifStmt = ifStmt
    self.elseStmt = elseStmt

  def print(self):
    self.expr.print()
    self.ifStmt.print()
    if self.elseStmt is not None:
      self.elseStmt.print()

class WhileStmt(Stmt):
  def __init__(self, expr, stmt):
    self.expr = expr
    self.stmt = stmt

  def print(self):
    self.expr.print()
    self.stmt.print()

class ReturnStmt(Stmt):
  def __init__(self, expr):
    self.expr = expr

  def print(self):
    self.expr.print()

class ReadIntStmt(Stmt):
  def __init__(self, id):
    self.id = id

  def print(self):
    print(id)

class WriteIntStmt(Stmt):
  def __init__(self, expr):
    self.expr = expr

  def print(self):
    self.expr.print()

class Expr(BaseBox):
  def __init__(self, id):
    self.id = id

  def print(self):
    print(self.id)


class ExprExpr(Expr):
  def __init__(self, id, expr):
    super().__init__(id)
    self.expr = expr

  def print(self):
    super().print()
    self.expr.print()


class ExprCond(BaseBox):
  def __init__(self, condition):
    self.condition = condition

  def print(self):
    self.condition.print()

class Condition(BaseBox):
  def __init__(self, disjunction):
    self.disjunction = disjunction

  def print(self):
    self.disjunction.print()

class ConditionExpr(Condition):
  def __init__(self, disjunction, expr, condition):
    super().__init__(disjunction)
    self.expr = expr
    self.condition = condition

  def print(self):
    super().print()
    self.expr.print()
    self.condition.print()


class Disjunction(BaseBox):
  def __init__(self, conjunction):
    self.conjunction = conjunction

  def print(self):
    self.conjunction.print()

class DisjunctionConjunction(Disjunction):
  def __init__(self, conjunction, disjunction):
    super().__init__(conjunction)
    self.disjunction = disjunction

  def print(self):
    super().print()
    self.disjunction.print()

class Conjunction(BaseBox):
  def __init__(self, comparison):
    self.comparison = comparison

  def print(self):
    self.comparison.print()

class ConjunctionComparison(Conjunction):
  def __init__(self, comparison, conjunction):
    super().__init__(comparison)
    self.conjunction = conjunction

  def print(self):
    super().print()
    self.conjunction.print()

class Comparison(BaseBox):
  def __init__(self, relation1, relation2=None):
    self.relation1 = relation1
    self.relation2 = relation2

  def print(self):
    self.relation1.print()
    if self.relation2 is not None:
      self.relation2.print()

class Relation(BaseBox):
  def __init__(self, sum):
    self.sum = sum

  def print(self):
    self.sum.print()

class RelationSum(Relation):
  def __init__(self, sum1, sum2):
    super().__init__(sum1)
    self.sum2 = sum2

  def print(self):
    super().print()
    self.sum2.print()

class Sum(BaseBox):
  def __init__(self, term):
    self.term = term

  def print(self):
    self.term.print()

class SumTerm(Sum):
  def __init__(self, sum, term):
    super().__init__(term)
    self.sum = sum

  def print(self):
    super().print()
    self.sum.print()

class Term(BaseBox):
  def __init__(self, factor):
    self.factor = factor

  def print(self):
    self.factor.print()


class TermFactor(Term):
  def __init__(self, term, factor):
    super().__init__(factor)
    self.term = term

  def print(self):
    super().print()
    self.term.print()

class Factor(BaseBox):
  def __init__(self, factor):
    self.factor = factor

  def print(self):
    self.factor.print()

class FactorPrimary(BaseBox):
  def __init__(self, primary):
    self.primary = primary


  def print(self):
    self.primary.print()

class Primary(BaseBox):
  def __init__(self, num=None, charconst=None, id=None, expr=None ):
    self.num = num
    self.charconst = charconst
    self.id = id
    self.expr = expr

  def print(self):
    if self.num is not None:
      print(self.num)
    elif self.charconst is not None:
      print(self.charconst)
    elif self.id is not None:
      print(self.id)
    elif expr is not None:
      self.expr.print()
    else:
      raise AssertionError('Primary -- Oops, this should not be possible!')

In [132]:
#implemente seu analisador sintático aqui
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['INT' , 'CHAR' , 
     'BREAK' , 'CONTINUE' ,
      'RETURN' ,
      'READINT' , 'WRITEINT' ,
      'IF' , 'ELSE',
      'WHILE' ,
      'ID' ,
      'OPEN_PARENS' , 'CLOSE_PARENS' ,
      'OPEN_COL' , 'CLOSE_COL' ,
      'VIRG' , 'PVIRG' ,
      'OPEN_CH' , 'CLOSE_CH' ,
      'COMPEQUALS' , 'COMPMAIOR' , 'COMPMENOR' ,
      'EQUALS' ,
      'INTERROG', 
      'DOISP', 'DISJ', 'CONJ',
      'NOT',
      'NUMBER', 'CHARCONST',
      'PLUS', 'MINUS', 'MUL', 'DIV', 'MOD'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV', 'MOD'])
    ]
)

@pg.error
def error_handler(token):
    raise ValueError("Token wasn't expected: " + token.name + " " + token.value + " " + str(token.source_pos))

# program ::= function (function)*
@pg.production('program : function')
def program(p):
  return Program(p[0])

@pg.production('program : function function')
def manyProgram(p):
  return Program(p[0], p[1])

# function ::= type_specifier id ‘(‘ param_decl_list ‘)’ compound_stmt
@pg.production('function : type_specifier ID OPEN_PARENS param_decl_list CLOSE_PARENS compound_stmt')
def function(p):
  return Function(p[0], p[1], p[3], p[5])

# type_specifier ::= int | char
@pg.production('type_specifier : INT')
@pg.production('type_specifier : CHAR')
def typeSpecifier(p):
  return p[0]

# param_decl_list ::= parameter_decl
@pg.production('param_decl_list : parameter_decl')
def paramDeclList(p):
    return ParamDeclList(p[0])

# param_decl_list ::= parameter_decl (‘,’ parameter_decl )*
@pg.production('param_decl_list : parameter_decl VIRG parameter_decl')
def manyParamDeclList(p):
    return ParamDeclList(p[0], p[2])

# param_decl ::= type_specifier ID
@pg.production('parameter_decl : type_specifier ID')
def paramDecl(p):
  return ParamDecl(p[0], p[1])

# compound_stmt ::= ‘{‘ (var_decl stmt)? ‘}’
@pg.production('compound_stmt : OPEN_CH var_decl stmt CLOSE_CH')
def compountStmt(p):
  return CompoundStmt(p[1], p[2])
    
# var_decl ::= type_specifier var_decl_list ‘;’
@pg.production('var_decl : type_specifier var_decl_list PVIRG')
def varDecl(p):
  return VarDecl(p[0], p[1])

#var_decl_list ::= variable_id ( ‘,’ variable_id)*
@pg.production('var_decl_list : variable_id')
def varDeclList(p):
    return VarDeclList(p[0])


#var_decl_list ::= variable_id ( ‘,’ variable_id)*
@pg.production('var_decl_list : variable_id VIRG variable_id')
def manyVarDeclList(p):
    return VarDeclList(p[0], p[2])


# variable_id ::= id ( ‘=’ expr )?
@pg.production('variable_id : ID')
def variableId(p):
  return VariableId(p[0])

@pg.production('variable_id : ID EQUALS expr')
def variableIdExpr(p):
  return VariableIdExpr(p[0], p[2])

# variable_id ::= id '[' num ']'
@pg.production('variable_id : ID OPEN_COL NUMBER CLOSE_COL')
def variableIdNum(p):
  return VariableIdNum(p[0], p[2])

# stmt ::= compound_stmt
@pg.production('stmt : compound_stmt')
def stmtCompound(p):
  return Stmt()

# stmt ::= cond_stmt
@pg.production('stmt : cond_stmt')
def stmtCond(p):
  return Stmt()

# stmt ::= while_stmt
@pg.production('stmt : while_stmt')
def stmtWhile(p):
  return Stmt()

# stmt ::= break ‘;’
@pg.production('stmt : BREAK PVIRG')
def stmtBreak(p):
  return Stmt()

# stmt ::= continue ‘;’
@pg.production('stmt : CONTINUE PVIRG')
def stmtContinue(p):
  return Stmt()

# stmt ::= return expr ‘;’
@pg.production('stmt : RETURN expr PVIRG')
def stmtReturnExpr(p):
  return ReturnStmt(p[1])

# stmt ::= readint ‘(‘ id ‘)’ ‘;’
@pg.production('stmt : READINT OPEN_PARENS ID CLOSE_PARENS PVIRG')
def stmtReadInt(p):
  return ReadIntStmt(p[2])

# stmt ::= writeint ‘(‘ expr ‘)’ ‘;’
@pg.production('stmt : WRITEINT OPEN_PARENS expr CLOSE_PARENS PVIRG')
def stmtWriteInt(p):
  return WriteIntStmt(p[2])

# cond_stmt ::= if ‘(‘ expr ‘)’ stmt
@pg.production('cond_stmt : IF OPEN_PARENS expr CLOSE_PARENS stmt')
def condStmtIf(p):
  return CondStmt(p[2], p[4])

# cond_stmt ::= if ‘(‘ expr ‘)’ stmt (else stmt)?
@pg.production('cond_stmt : IF OPEN_PARENS expr CLOSE_PARENS stmt ELSE stmt')
def condStmtIfElse(p):
  return CondStmt(p[2], p[4], p[6])


# while_stmt ::= while ‘(‘ expr ‘)’ stmt
@pg.production('while_stmt : WHILE OPEN_PARENS expr CLOSE_PARENS stmt')
def whileStmt(p):
  return WhileStmt(p[2], p[4])

# expr ::= id ‘=’ expr
@pg.production('expr : ID EQUALS expr')
def exprExpr(p):
  return ExprExpr(p[0], p[2])

# expr ::= condition
@pg.production('expr : condition')
def exprCond(p):
  return ExprCond(p[0])


# condition ::= disjunction
@pg.production('condition : disjunction')
def condition(p):
  return Condition(p[0])

# condition : disjunction ‘?’ expr ‘:’ condition
@pg.production('condition : disjunction INTERROG  expr DOISP condition')
def conditionExpr(p):
  return ConditionExpr(p[0], p[2], p[4])

#disjunction ::= conjunction | disjunction ‘||’ conjunction
@pg.production('disjunction : conjunction')
def disjunction(p):
  return Disjunction(p[0])

#disjunction ::= disjunction ‘||’ conjunction
@pg.production('disjunction : disjunction DISJ conjunction')
def disjunctionConjunction(p):
  return DisjunctionConjunction(p[2], p[0])

#conjunction ::= comparison
@pg.production('conjunction : comparison')
def conjunction(p):
  return Conjunction(p[0])

#conjunction ::= conjunction ‘&&’ comparison
@pg.production('conjunction : conjunction CONJ comparison')
def conjunction(p):
  return ConjunctionComparison(p[2], p[0])

#comparison ::= relation 
@pg.production('comparison : relation')
def comparison(p):
  return Comparison(p[0])

#comparison ::=  relation ‘==’ relation
@pg.production('comparison : relation COMPEQUALS relation')
def comparisonEquals(p):
  return Comparison(p[0], p[2])

#relation ::= sum 
@pg.production('relation : sum')
def relationSum(p):
  return Relation(p[0])

#relation ::= sum ‘<’ sum
@pg.production('relation : sum COMPMENOR sum')
def relationSumLessThan(p):
  return RelationSum(p[0], p[2])

#relation ::= sum ‘>’ sum
@pg.production('relation : sum COMPMAIOR sum')
def relationSumGreatherThan(p):
  return RelationSum(p[0], p[2])
 
#sum ::= term
@pg.production('sum : term')
def sum(p):
  return Sum(p[0])

#sum ::= sum ‘+’ term
@pg.production('sum : sum PLUS term')
def sumTermPlus(p):
  return SumTerm(p[0], p[2])

#sum ::= sum ‘-’ term
@pg.production('sum : sum MINUS term')
def sumTermMinus(p):
  return SumTerm(p[0], p[2])

#term ::= factor
@pg.production('term : factor')
def term(p):
  return Term(p[0])

#term ::= term ‘*’ factor 
@pg.production('term : term MUL factor')
def termFactorMult(p):
  return TermFactor(p[0], p[2])

#term ::=  term ‘/’ factor 
@pg.production('term : term DIV factor')
def termFactorDiv(p):
  return TermFactor(p[0], p[2])

#term ::=  term ‘%’ factor 
@pg.production('term : term MOD factor')
def termFactorMod(p):
  return TermFactor(p[0], p[2])

#factor ::= primary
@pg.production('factor : primary')
def fatorPrim(p):
  return FactorPrimary(p[0])

#factor ::= ‘!’ factor
@pg.production('factor : NOT factor')
def notFator(p):
  return Factor(p[1])

#factor ::= ‘-’ factor
@pg.production('factor : MINUS factor')
def minusFator(p):
  return Factor(p[1])

#primary ::= num
@pg.production('primary : NUMBER')
def primaryNum(p):
  return Primary(num=p[0])

#primary ::= charconst
@pg.production('primary : CHARCONST')
def primaryChar(p):
  return Primary(charconst=p[0])

#primary ::= id
@pg.production('primary : ID')
def primaryId(p):
  return Primary(id=p[0])

#primary ::= ‘(‘ expr ‘)’ 
@pg.production('primary : OPEN_PARENS expr CLOSE_PARENS')
def primaryExpr(p):
  return Primary(expr=p[1])

parser = pg.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:300: ParserGeneratorWarning: 1 shift/reduce conflict


In [133]:
#teste seu analisador com um pequeno programa em TINY-C.
arvoreTinyC=parser.parse(lexer.lex('int sum(int a, int b){int soma = a + b;  return soma;}'))
print(arvoreTinyC.print())

Token('INT', 'int')
Token('ID', 'sum')
Token('INT', 'int')
Token('ID', 'a')
Token('INT', 'int')
Token('ID', 'b')
Token('INT', 'int')
Token('ID', 'soma')
Token('ID', 'b')
Token('ID', 'a')
Token('ID', 'soma')
None


**ATIVIDADE EAD**

Finalizar a implementação do analisador sintático para TINY-C.

